# Capítulo 2: Carga de datos

Aquí se lee el dataset elegido de **Datos Abiertos** (CHC_2021) en formato CSV/XLS/XLSX y se guarda una copia de trabajo en **Parquet** para acelerar cargas posteriores.

**Objetivo:** obtener una versión limpia y eficiente del dataset de entrada.  
**Entradas:** archivo de CHC_2021 (`.csv`/`.xlsx`).  
**Salidas:** `data/CHC_2021_raw.parquet` (sin transformar aún).


In [2]:
import pandas as pd
import geopandas as gpd
import os


In [5]:
import os
import pandas as pd
import geopandas as gpd

# Rutas
RUTA_SHAPE = r"C:\Users\sergi\Downloads\MGN2021_DPTO_POLITICO"   # <-- ajusta con la ruta real
RUTA_DATA  = r"C:\Users\sergi\Downloads\CHC_2021\CHC_base_anonimizada09-09-2021.csv"

# Cargar shapefile
gdf_depto = gpd.read_file(RUTA_SHAPE).to_crs(4326)
gdf_depto = gdf_depto.rename(columns={
    'DPTO_CCDGO':'DPTO_CCDGO',
    'DPTO_CNMBR':'DPTO_CNMBR'
})[['DPTO_CCDGO','DPTO_CNMBR','geometry']]
print("Shapefile cargado:", gdf_depto.shape)
display(gdf_depto.head())

# Cargar dataset (CSV o Excel)
ext = os.path.splitext(RUTA_DATA)[1].lower()
if ext == ".csv":
    df = pd.read_csv(RUTA_DATA, encoding="latin-1", sep=None, engine="python")
elif ext in (".xlsx", ".xls"):
    df = pd.read_excel(RUTA_DATA)
else:
    raise ValueError("Usa CSV o XLSX para RUTA_DATA")

print("Dataset cargado:", df.shape)
df.head()


Shapefile cargado: (33, 3)


,DPTO_CCDGO,DPTO_CNMBR,geometry
0,05,ANTIOQUIA,"POLYGON ((-76.41355 8.87383, -76.40465 8.85195..."
1,08,ATLÁNTICO,"POLYGON ((-74.84946 11.09778, -74.84938 11.097..."
2,11,"BOGOTÁ, D.C.","POLYGON ((-74.07059 4.82856, -74.07036 4.82856..."
3,13,BOLÍVAR,"MULTIPOLYGON (((-76.17318 9.38785, -76.17287 9..."
4,15,BOYACÁ,"POLYGON ((-72.17368 7.05308, -72.17277 7.05224..."


Dataset cargado: (6250, 130)


,DIRECTORIO,TIP_FOR,P1,P1S1,P2,P2S1,P5,CTL_1,P8R,P9,...,P35,P36R,P37S1,P37S2,P37S3,P37S4,P37S5,P37S6,P37S7,COMPLETA
0,1.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,44.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,33.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,3.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,41.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,4.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,29.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,5.0,2.0,85.0,85001.0,1.0,5.0,2.0,1.0,30.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Carga de datos

Se cargaron dos fuentes de información:

1. **Shapefile de departamentos**  
   Contiene 33 filas y 3 columnas principales:
   - `DPTO_CCDGO`: código del departamento.
   - `DPTO_CNMBR`: nombre del departamento.
   - `geometry`: geometría asociada (polígonos o multipolígonos).

   Ejemplo de las primeras filas:

   | DPTO_CCDGO | DPTO_CNMBR | geometry |
   |------------|------------|----------|
   | 05 | ANTIOQUIA | POLYGON(...) |
   | 08 | ATLÁNTICO | POLYGON(...) |
   | 11 | BOGOTÁ, D.C. | POLYGON(...) |
   | 13 | BOLÍVAR | MULTIPOLYGON(...) |
   | 15 | BOYACÁ | POLYGON(...) |

---

2. **Base de datos principal (DANE: economía y sociedad)**  
   Contiene **6.250 filas y 130 columnas**.  
   Entre las variables se incluyen:

   - `DIRECTORIO`: identificador del hogar.
   - `TIP_FOR`: tipo de formulario.
   - `P1`, `P1S1`, `P2`, `P2S1`, etc.: variables de encuesta.
   - Otras columnas (`P35`, `P36R`, `P37S1`, …) con valores numéricos o faltantes (`NaN`).

   Ejemplo de las primeras filas:

   | DIRECTORIO | TIP_FOR | P1  | P1S1  | P2  | ... | P35 | P36R | P37S1 |
   |------------|---------|-----|-------|-----|-----|-----|------|-------|
   | 1.0 | 2.0 | 85.0 | 85001.0 | 1.0 | ... | NaN | NaN  | NaN   |
   | 2.0 | 2.0 | 85.0 | 85001.0 | 1.0 | ... | NaN | NaN  | NaN   |
   | 3.0 | 2.0 | 85.0 | 85001.0 | 1.0 | ... | NaN | NaN  | NaN   |

---

### Observaciones iniciales
- El **shapefile** servirá para representar geográficamente la información de los departamentos.  
- El **dataset** contiene la información estadística a analizar y posteriormente podrá vincularse al shapefile mediante el código de departamento (`DPTO_CCDGO` ↔ variables del dataset).
- Existen columnas con valores faltantes (`NaN`), que deberán ser tratadas en la limpieza de datos.


In [7]:
import os
os.makedirs("../data", exist_ok=True)

# Guarda en dos formatos para no fallar:
df.to_parquet("../data/CHC_2021_raw.parquet", index=False)
df.to_csv("../data/CHC_2021_raw.csv", index=False, encoding="utf-8")

print("Guardados: CHC_2021_raw.parquet y CHC_2021_raw.csv en ../data")



Guardados: CHC_2021_raw.parquet y CHC_2021_raw.csv en ../data
